In [5]:
import requests
from urllib.parse import quote
from google.colab import files
import json

import pandas as pd
import folium

In [6]:
filename = './kakaoAPI.txt'
with open(filename) as f:
   api_key = f.read()

In [7]:
print(len(api_key))

32


In [8]:
addr = '서울특별시 종로구 종로1길 36(수송동)'
quote(addr)

'%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C%20%EC%A2%85%EB%A1%9C%EA%B5%AC%20%EC%A2%85%EB%A1%9C1%EA%B8%B8%2036%28%EC%88%98%EC%86%A1%EB%8F%99%29'

In [9]:
search_url = "https://dapi.kakao.com/v2/local/search/address.json"
url = f'{search_url}?query={quote(addr)}'
url

# "https://dapi.kakao.com/v2/local/search/address.json" \
#   -H "Authorization: KakaoAK ${REST_API_KEY}" \
#   --data-urlencode "query=전북 삼성동 100" 

'https://dapi.kakao.com/v2/local/search/address.json?query=%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C%20%EC%A2%85%EB%A1%9C%EA%B5%AC%20%EC%A2%85%EB%A1%9C1%EA%B8%B8%2036%28%EC%88%98%EC%86%A1%EB%8F%99%29'

In [10]:
# get map infos from kakao 
result = requests.get(url, headers={"Authorization": f'KakaoAK {api_key}'}).json()

In [11]:
result

{'documents': [{'address': {'address_name': '서울 종로구 수송동 146-12',
    'b_code': '1111012400',
    'h_code': '1111061500',
    'main_address_no': '146',
    'mountain_yn': 'N',
    'region_1depth_name': '서울',
    'region_2depth_name': '종로구',
    'region_3depth_h_name': '종로1.2.3.4가동',
    'region_3depth_name': '수송동',
    'sub_address_no': '12',
    'x': '126.978988255925',
    'y': '37.5735051436739'},
   'address_name': '서울 종로구 종로1길 36',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '서울 종로구 종로1길 36',
    'building_name': '종로구청',
    'main_building_no': '36',
    'region_1depth_name': '서울',
    'region_2depth_name': '종로구',
    'region_3depth_name': '수송동',
    'road_name': '종로1길',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '126.978988255925',
    'y': '37.5735051436739',
    'zone_no': '03152'},
   'x': '126.978988255925',
   'y': '37.5735051436739'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [12]:
# json() 삭제후 확인 -> 200
# result.status_code

In [13]:
# json()으로 받지 않은 경우: result.txt
# result.txt
# res = json.loads(result.txt)

In [14]:
result.keys()

dict_keys(['documents', 'meta'])

In [15]:
result['documents'][0]['x']
result['documents'][0]['y']

'37.5735051436739'

In [16]:
lng = float(result['documents'][0]['x'])
lat = float(result['documents'][0]['y'])

In [17]:
df = pd.read_csv('./공공기관.csv')
df

,공공기관,도로명주소
0,종로구청,서울특별시 종로구 종로1길 36(수송동)
1,노원구청,서울특별시 노원구 노해로 437(상계동)
2,송파구청,서울특별시 송파구 올림픽로 326(신천동)
3,마포구청,서울특별시 마포구 월드컵로 지하190(성산동)
4,양천구청,서울특별시 양천구 목동동로 105(신정동)


In [18]:
lat_list = []
lng_list = []

In [19]:
# get lat, lng from address
for i in df.index:
   search_url = "https://dapi.kakao.com/v2/local/search/address.json"
   url = f'{search_url}?query={quote(df.도로명주소[i])}'
   result = requests.get(url, headers={"Authorization": f'KakaoAK {api_key}'}).json()
   lng = float(result['documents'][0]['x'])
   lat = float(result['documents'][0]['y'])
   lat_list.append(lat)
   lng_list.append(lng)

In [20]:
df['위도'] = lat_list
df['경도'] = lng_list
df.to_csv('공공기관2.csv', index=False)

In [23]:
df

,공공기관,도로명주소,위도,경도
0,종로구청,서울특별시 종로구 종로1길 36(수송동),37.573505,126.978988
1,노원구청,서울특별시 노원구 노해로 437(상계동),37.654523,127.056268
2,송파구청,서울특별시 송파구 올림픽로 326(신천동),37.514477,127.105860
3,마포구청,서울특별시 마포구 월드컵로 지하190(성산동),37.563426,126.903357
4,양천구청,서울특별시 양천구 목동동로 105(신정동),37.517075,126.866543


In [21]:
lat_list

[37.5735051436739,
 37.6545228397157,
 37.514477182474,
 37.5634263131969,
 37.5170753784215]

In [40]:
map = folium.Map(location = [df.위도.mean(), df.경도.mean()], zoom_start=12)

for i in df.index:
   folium.Circle(radius=300,
                 location=(df.위도[i], df.경도[i]),
                           popup = folium.Popup(df.도로명주소[i], max_width=200),
                           tooltip = df.공공기관[i],
                           color = 'red',
                           fill = True,
                           fill_color = '#3186cc').add_to(map)

# create title
title = '<h3 align = "center" style = "font-size:20">서울시 구청</h3>'
map.get_root().html.add_child(folium.Element(title))
map